In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import Isomap
from sklearn.cluster import KMeans
import matplotlib.patches as mpatches
from scipy.io import loadmat, savemat
import networkx as nx
import community as community_louvain
import os

In [3]:
def fisher_r_to_z(r):
    return np.arctanh(r)

def load_mat_file(file_path):
    # Load the .mat file
    mat = loadmat(file_path)
    return mat['data'] 

def save_average_correlation_matrix(subject_id, contrast_id, avg_corr_matrix):
    # Save the average correlation matrix to a CSV file
    df = pd.DataFrame(avg_corr_matrix)
    df.to_csv(f'average_matrices/{subject_id}_contrast_{contrast_id}_average_correlation.csv', index=False)

def load_average_correlation_matrix(subject_id, contrast_id):
    # Load the average correlation matrix from a CSV file
    try:
        df = pd.read_csv(f'average_matrices/{subject_id}_contrast_{contrast_id}_average_correlation.csv')
        return df.values
    except FileNotFoundError:
        return None

def create_and_save_graph(subject_id, contrast_id, correlation_matrix, correlation_threshold=0.3):
    if correlation_matrix is None:
        print(f"Skipping graph creation for subject {subject_id}, contrast {contrast_id} due to missing data.")
        return

    num_channels = correlation_matrix.shape[0]
    
    G = nx.Graph()
    for j in range(num_channels):
        for k in range(j + 1, num_channels):
            if correlation_matrix[j, k] > correlation_threshold:
                G.add_edge(j, k, weight=correlation_matrix[j, k])
    
    nx.write_graphml(G, f'graphs/{subject_id}_contrast_{contrast_id}_network.graphml')

def main():
    file_path = 'dataMSc.mat'
    data = load_mat_file(file_path)
    
    num_subjects = data.shape[1]
    num_contrasts = 3  
    num_channels = 64 
    
    os.makedirs('average_matrices', exist_ok=True)
    os.makedirs('graphs', exist_ok=True)
    
    for i in range(num_subjects):
        subject_data = data[0, i]
        num_runs = len(subject_data[0])
        
        if num_runs == 0 or subject_data is None or all(run.size == 0 for run in subject_data[0]):
            print(f"Skipping subject {i + 1} due to missing or empty data.")
            continue  # No data for this subject

        average_correlation_matrices = np.zeros((num_channels, num_channels, num_contrasts))
        
        for contrast in range(num_contrasts):
            z_sum = np.zeros((num_channels, num_channels))
            count = np.zeros((num_channels, num_channels))
            
            for run in range(num_runs):
                run_data = subject_data[0, run]
                
                if run_data.size == 0:
                    continue  # Empty run data
                
                correlation_matrix = run_data[:, :, contrast]
                valid = ~np.isnan(correlation_matrix)  # a mask to identify indices with not (~) NaN values
                
                z_values = np.zeros_like(correlation_matrix)
                z_values[valid] = fisher_r_to_z(correlation_matrix[valid])
                
                z_sum[valid] += z_values[valid]
                count[valid] += 1
            
            # Avoid division by zero
            valid_counts = count > 0
            z_avg = np.zeros_like(z_sum)
            z_avg[valid_counts] = z_sum[valid_counts] / count[valid_counts]
            
            average_correlation_matrices[:, :, contrast] = z_avg

        for contrast in range(num_contrasts):
            save_average_correlation_matrix(f'{i + 1}', contrast + 1, average_correlation_matrices[:, :, contrast])

    # Now create and save graphs from the average correlation matrices
    for i in range(num_subjects):
        for contrast in range(num_contrasts):
            avg_corr_matrix = load_average_correlation_matrix(f'{i + 1}', contrast + 1)
            create_and_save_graph(f'{i + 1}', contrast + 1, avg_corr_matrix)

if __name__ == "__main__":
    main()

Skipping subject 1 due to missing or empty data.
Skipping subject 2 due to missing or empty data.
Skipping subject 3 due to missing or empty data.
Skipping subject 4 due to missing or empty data.
Skipping subject 5 due to missing or empty data.
Skipping subject 6 due to missing or empty data.
Skipping subject 7 due to missing or empty data.
Skipping subject 8 due to missing or empty data.
Skipping subject 9 due to missing or empty data.
Skipping subject 10 due to missing or empty data.
Skipping subject 11 due to missing or empty data.
Skipping subject 23 due to missing or empty data.
Skipping subject 27 due to missing or empty data.
Skipping subject 28 due to missing or empty data.
Skipping subject 29 due to missing or empty data.
Skipping subject 30 due to missing or empty data.
Skipping subject 31 due to missing or empty data.
Skipping subject 36 due to missing or empty data.
Skipping graph creation for subject 1, contrast 1 due to missing data.
Skipping graph creation for subject 1,